# Maximum Likelihood Expectation Maximization
Frequentist optimization of liklelihood surface

In [1]:
import lzma
from pathlib import Path

import dill as pickle
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
import ipyparallel as ipp

In [3]:
from scipy.optimize import minimize

In [4]:
import elm

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [5]:
import rxmc

In [6]:
evidence_dir = Path("../evidence/data")
model_dir = Path("../model/")
likelihood_model_dir = Path("../likelihood_model/")
prior_path = Path("../prior/prior_distribution.pickle")

In [7]:
output_dir = Path("./elm_prior_propagation//")
output_dir.mkdir(parents=True, exist_ok=True)

## read in ELM prior parameter samples

In [8]:
with open(prior_path, "rb") as f:
    prior = pickle.load(f)
prior_samples = prior.rvs(1000)

## read in observations

In [9]:
with lzma.open(evidence_dir / "nn_elastic_observations.xz", "rb") as f:
    nn_elastic_obs = pickle.load(f)
with lzma.open(evidence_dir / "pp_elastic_observations.xz", "rb") as f:
    pp_elastic_obs = pickle.load(f)

## read in physical models

In [10]:
with open(model_dir / "elm_nn_model.pkl", "rb") as f:
    elm_nn_model = pickle.load(f)

In [11]:
with open(model_dir / "elm_pp_model.pkl", "rb") as f:
    elm_pp_model = pickle.load(f)

## read in likelihood models

In [12]:
with open(likelihood_model_dir / "likelihood_model.pkl", "rb") as f:
    likelihood_model = pickle.load(f)

## Set up Evidence object

In [13]:
pp_constraints = [
    rxmc.constraint.Constraint(
        observations=obs,
        physical_model=elm_pp_model,
        likelihood_model=likelihood_model,
    )
    for entry_id, obs in pp_elastic_obs.items()
]
nn_constraints = [
    rxmc.constraint.Constraint(
        observations=obs,
        physical_model=elm_nn_model,
        likelihood_model=likelihood_model,
    )
    for entry_id, obs in nn_elastic_obs.items()
]

In [14]:
evidence = rxmc.evidence.Evidence(constraints=nn_constraints + pp_constraints)

## Run MLEM

In [15]:
bounds = [p.bounds for p in elm.params]
bounds

[(30, 80),
 (0, 30),
 (20, 50),
 (0, 60),
 (-10, 60),
 (-10, 60),
 (-0.4, -0.3),
 (-0.01, 0.01),
 (10, 60),
 (10, 60),
 (0.9, 1.5),
 (0.9, 1.5),
 (0.4, 1.0),
 (0.4, 1.0)]

In [16]:
good_samples = np.concatenate(
    [np.load("/home/beyerk/db/chex_calibration/results/qe_cal_samples.npy")]
)
mask = np.ones(15, dtype=bool)
mask[6] = False
good_starting_point = np.mean(good_samples[:, mask], axis=0)
good_ssample_stdev = np.std(good_samples[:, mask], axis=0)

In [ ]:
%%timeit
evidence.log_likelihood(prior.mean)

In [ ]:
%%time
result = minimize(
    lambda x: -evidence.log_likelihood(x, executor=rc),
    good_starting_point,
    bounds=bounds,
)

In [21]:
with open("mlem_result.pkl", "wb") as f:
    pickle.dump(result, f)

NameError: name 'result' is not defined

In [ ]:
result

In [ ]:
for p, x in zip(elm.params, result.x):
    print(p.name, x)